### Automated backwards optimization

Automate the discovery of strategy rules.

In [ ]:
generate indicator combinations
    * [2, 4, 8, etc]
    * [adx, ]

for i in indicators:
    for start_date in range(first_date, last_date):
        for end_date in range(start_date, last_date):
            for lower_limit in range(min(i), max(i), some_step):
                for upper_limit in range(lower_limit + 1?, max(i), some_step):

                    compute profitability in range

                    save results

for i, j in indicator pairs:
    for start_date in range(first_date, last_date):
        for end_date in range(start_date, last_date):
            for lower_limit_i in range(min(i), max(i), some_step):
                for upper_limit_i in range(lower_limit + 1?, max(i), some_step):                    
                    for lower_limit_j in range(min(j), max(j), some_step):
                        for upper_limit_j in range(lower_limit + 1?, max(j), some_step):
                            compute profitability in range

                            save results


In [60]:
import datetime
import json
from datetime import timedelta
from dateutil.relativedelta import relativedelta
from dateutil import rrule
from itertools import product, combinations

import numpy as np

import pandas as pd
import tqdm
from matplotlib import pyplot
import seaborn as sns
import dask.dataframe as dd

# TIMEFRAME = 'daily'
TIMEFRAME = "weekly"

# FRAMEWORK = 'dask'
FRAMEWORK = "pandas"

FIRST_DATE = datetime.datetime(2000, 1, 1)
LAST_DATE = datetime.datetime(2020, 12, 31)

FORWARD_OPT_RESULTS_JSON = f"results/{TIMEFRAME}/forward_opt_results.json"
BACKWARD_OPT_RESULTS_JSON = f"results/{TIMEFRAME}/backward_opt_results.json"

# The minimum number of trades in a period for it to be considered.
MIN_TRADES_REQUIRED = 10

In [ ]:
metrics_options = ["adx", "ppo", "stochastic"]


if TIMEFRAME == "daily":
    size_options = [8, 16, 32]
    path = "dataset/daily/"
    column_names = [
        "uid",
        "ticker",
        "date",
        "price_open",
        "maperiod",
        "rsi_open_period",
        "adx8",
        "adx16",
        "adx32",
        "ppo8",
        "ppo16",
        "ppo32",
        "stochastic8",
        "stochastic16",
        "stochastic32",
        "price_sell",
        "days_ago_close_period",
        "rsi_close_period",
    ]
elif TIMEFRAME == "weekly":
    size_options = [3, 6, 9]
    path = "dataset/weekly/"
    column_names = [
        "uid",
        "ticker",
        "date",
        "price_open",
        "maperiod",
        "rsi_open_period",
        "adx3",
        "adx6",
        "adx9",
        "ppo3",
        "ppo6",
        "ppo9",
        "stochastic3",
        "stochastic6",
        "stochastic9",
        "price_sell",
        "days_ago_close_period",
        "rsi_close_period",
    ]

if FRAMEWORK == "pandas":
    d = pd
    f = f"{path}/all_results.csv"
elif FRAMEWORK == "dask":
    d = dd
    f = f"{path}/*.csv"

FULL_DF = d.read_csv(
    f,
    names=column_names,
)


def get_full_df():
    return d.read_csv(
        f,
        names=column_names,
    )

In [ ]:
get_full_df()

In [61]:
def get_forward_opt_json():
    with open(FORWARD_OPT_RESULTS_JSON, "r") as f:
        return json.loads(f.read())


FORWARD_OPT_JSON = get_forward_opt_json()


def get_filtered_df_by_key(key, range_name, func_type):
    optimal_args = FORWARD_OPT_JSON[key][range_name][func_type]
    df = get_full_df()

    for k, v in optimal_args.items():
        df = df[df[k] == v]

    return df


# get_filtered_df_by_key('17-10', '6-month', 'mean')

In [62]:
keys = [
    end_date.strftime("%y-%m")
    for end_date in rrule.rrule(rrule.MONTHLY, dtstart=FIRST_DATE, until=LAST_DATE)
]
range_values = [
    "6-month",
    "12-month",
    "max",
]
func_values = ["mean", "order"]

In [63]:
def get_backward_opt_json():
    with open(BACKWARD_OPT_RESULTS_JSON, "r") as f:
        return json.loads(f.read())


BACKWARD_OPT_JSON = get_backward_opt_json()


def is_pair_parsed(key, range_val, func_val):
    if key not in BACKWARD_OPT_JSON:
        return False

    if range_val not in BACKWARD_OPT_JSON[key]:
        return False

    if func_val not in BACKWARD_OPT_JSON[key][range_val]:
        return False

    return True


# for key, range_val, func_val in list(product(keys, range_values, func_values)):
#     if is_pair_parsed(key, range_val, func_val) is False:
#         print(f"Pair {key} - {range_val} - {func_val} is NOT already parsed!")

In [64]:
def save_pair_result(key, range_val, func_val, optimal_res):
    with open(BACKWARD_OPT_RESULTS_JSON, "r") as f:
        backward_opt_json = json.loads(f.read())

    backward_opt_json[key] = backward_opt_json.get(key, {})
    backward_opt_json[key][range_val] = backward_opt_json[key].get(range_val, {})
    backward_opt_json[key][range_val][func_val] = optimal_res

    with open(BACKWARD_OPT_RESULTS_JSON, "w") as f:
        f.write(json.dumps(backward_opt_json))


# save_pair_result('00-01', '6-month', 'mean', 1)
# is_pair_parsed('00-01', '6-month', 'mean')

In [65]:
def mean_optimal_func(tmp_df):
    max_value = tmp_df["perc_change"].mean()

    if FRAMEWORK == "dask":
        max_value = max_value.compute()

    return max_value


def order_optimal_func(tmp_df):
    l = []
    for _, perc_change in tmp_df.sort_values(["date"])["perc_change"].iteritems():
        l.append(perc_change)

    results = []
    for i in range(len(l)):
        res = 1
        for j in range(i, len(l)):
            res += (res * l[j]) / 100
        results.append(res)

    return sum(results) / len(results)

In [75]:
singles = [f"{x[0]}{x[1]}" for x in list(product(metrics_options, size_options))]
# doubles = list(combinations(indicators, 2))


def single_compute_optimal_res(df, func):
    upper_barrier_results = []
    lower_barrier_results = []

    for indicator in singles:
        max_val, min_val = int(df[indicator].max()), int(df[indicator].min())

        for barrier in range(min_val, max_val + 2, 2):
            filtered_df = df[df[indicator] <= barrier]
            if filtered_df.shape[0] >= MIN_TRADES_REQUIRED:
                outcome = func(filtered_df)
                if outcome and outcome not in [np.nan]:
                    upper_barrier_results.append([indicator, barrier, float(outcome)])

            filtered_df = df[df[indicator] >= barrier]
            if filtered_df.shape[0] >= MIN_TRADES_REQUIRED:
                outcome = func(filtered_df)
                if outcome and outcome not in [np.nan]:
                    lower_barrier_results.append([indicator, barrier, float(outcome)])

    upper_barrier_df = pd.DataFrame(
        np.array(upper_barrier_results), columns=["indicator", "barrier", "value"]
    )
    upper_barrier_res = upper_barrier_df[
        upper_barrier_df["value"] == upper_barrier_df["value"].max()
    ]

    lower_barrier_df = pd.DataFrame(
        np.array(lower_barrier_results), columns=["indicator", "barrier", "value"]
    )
    lower_barrier_res = lower_barrier_df[
        lower_barrier_df["value"] == lower_barrier_df["value"].max()
    ]

    if upper_barrier_df["value"].max():
        best_type = "upper"
        best_indicator = upper_barrier_res.iloc[0]["indicator"]
        best_barrier = int(upper_barrier_res.iloc[0]["barrier"])
    else:
        best_type = "lower"
        best_indicator = lower_barrier_res.iloc[0]["indicator"]
        best_barrier = int(lower_barrier_res.iloc[0]["barrier"])

    return {
        "upper": {
            "indicator": upper_barrier_res.iloc[0]["indicator"],
            "barrier": int(upper_barrier_res.iloc[0]["barrier"]),
        },
        "lower": {
            "indicator": lower_barrier_res.iloc[0]["indicator"],
            "barrier": int(lower_barrier_res.iloc[0]["barrier"]),
        },
        "best": {
            "type": best_type,
            "indicator": best_indicator,
            "barrier": best_barrier,
        },
    }


def double_compute_optimal_res(df, func):
    pass


def compute_optimal_res(df):
    result = {}
    for name, func in [("mean", mean_optimal_func), ("order", order_optimal_func)]:
        result[name] = {
            "single": single_compute_optimal_res(df, func),
            #             'double': double_compute_optimal_res(df, func),
        }

    return result

In [69]:
all_pairs = list(product(keys, range_values, func_values))

unparsed_pairs = [
    (key, range_val, func_val)
    for key, range_val, func_val in all_pairs
    if not is_pair_parsed(key, range_val, func_val)
]

len(unparsed_pairs)

1050

In [ ]:
with tqdm.tqdm(total=len(unparsed_pairs)) as pbar:

    for key, range_val, func_val in unparsed_pairs:
        try:
            df = get_filtered_df_by_key(key, range_val, func_val)

            df["perc_change"] = (
                (df["price_sell"] - df["price_open"]) / df["price_open"] * 100
            )
            df = df.drop(["price_sell", "price_open"], axis=1)
            print(
                f"Key in forward opt json {key} - {range_val} - {func_val} - {df.shape[0]}"
            )

        except KeyError:
            print(f"Key NOT in forward opt json {key} - {range_val} - {func_val}")
            pbar.update()
            continue

        size = df.shape[0] if FRAMEWORK == "pandas" else df.shape[0].compute()

        if size < MIN_TRADES_REQUIRED:
            print(f"Not enough trades for {key} - {range_val} - {func_val}")
            optimal_res = None
        else:
            optimal_res = compute_optimal_res(df)
        # 06-06 - 6-month - mean - 10
        print(f"{key} - {range_val} - {func_val}")
        print(optimal_res)
        save_pair_result(key, range_val, func_val, optimal_res)
        pbar.update()

  0%|          | 1/1050 [00:00<04:50,  3.61it/s]

Key in forward opt json 06-06 - 6-month - mean - 10
06-06 - 6-month - mean
{'mean': {'single': {'upper': {'indicator': 'adx3', 'barrier': 69}, 'lower': {'indicator': 'adx3', 'barrier': 55}, 'best': {'type': 'upper', 'indicator': 'adx3', 'barrier': 69}}}, 'order': {'single': {'upper': {'indicator': 'adx3', 'barrier': 69}, 'lower': {'indicator': 'adx3', 'barrier': 55}, 'best': {'type': 'upper', 'indicator': 'adx3', 'barrier': 69}}}}
Key in forward opt json 06-06 - 6-month - order - 58


  0%|          | 2/1050 [00:00<08:27,  2.07it/s]

06-06 - 6-month - order
{'mean': {'single': {'upper': {'indicator': 'adx6', 'barrier': 34}, 'lower': {'indicator': 'ppo3', 'barrier': 5}, 'best': {'type': 'upper', 'indicator': 'adx6', 'barrier': 34}}}, 'order': {'single': {'upper': {'indicator': 'adx3', 'barrier': 63}, 'lower': {'indicator': 'ppo3', 'barrier': 5}, 'best': {'type': 'upper', 'indicator': 'adx3', 'barrier': 63}}}}
Key in forward opt json 06-06 - 12-month - mean - 795


  0%|          | 3/1050 [00:08<1:03:51,  3.66s/it]

...

13-02 - 6-month - mean
{'mean': {'single': {'upper': {'indicator': 'adx3', 'barrier': 49}, 'lower': {'indicator': 'ppo6', 'barrier': 11}, 'best': {'type': 'upper', 'indicator': 'adx3', 'barrier': 49}}}, 'order': {'single': {'upper': {'indicator': 'adx3', 'barrier': 67}, 'lower': {'indicator': 'stochastic3', 'barrier': 7}, 'best': {'type': 'upper', 'indicator': 'adx3', 'barrier': 67}}}}
Key in forward opt json 13-02 - 6-month - order - 842


 46%|████▌     | 482/1050 [46:04<1:05:14,  6.89s/it]

13-02 - 6-month - order
{'mean': {'single': {'upper': {'indicator': 'ppo6', 'barrier': 0}, 'lower': {'indicator': 'ppo9', 'barrier': 17}, 'best': {'type': 'upper', 'indicator': 'ppo6', 'barrier': 0}}}, 'order': {'single': {'upper': {'indicator': 'stochastic3', 'barrier': 36}, 'lower': {'indicator': 'ppo9', 'barrier': 7}, 'best': {'type': 'upper', 'indicator': 'stochastic3', 'barrier': 36}}}}
Key in forward opt json 13-02 - 12-month - mean - 843


 46%|████▌     | 483/1050 [46:15<1:16:05,  8.05s/it]

13-02 - 12-month - mean
{'mean': {'single': {'upper': {'indicator': 'ppo3', 'barrier': -2}, 'lower': {'indicator': 'ppo6', 'barrier': 20}, 'best': {'type': 'upper', 'indicator': 'ppo3', 'barrier': -2}}}, 'order': {'single': {'upper': {'indicator': 'stochastic6', 'barrier': 85}, 'lower': {'indicator': 'stochastic3', 'barrier': 14}, 'best': {'type': 'upper', 'indicator': 'stochastic6', 'barrier': 85}}}}
Key in forward opt json 13-02 - 12-month - order - 843


In [49]:
# 06-06 - 6-month - mean - 10
tmpdf = get_filtered_df_by_key("00-01", "6-month", "mean")
tmpdf["perc_change"] = (
    (tmpdf["price_sell"] - tmpdf["price_open"]) / tmpdf["price_open"] * 100
)
tmpdf = tmpdf.drop(["price_sell", "price_open"], axis=1)

In [50]:
tmpdf["perc_change"].describe()

count    225.000000
mean       1.380352
std        9.567881
min      -75.889165
25%       -2.319029
50%        2.077950
75%        5.875300
max       30.632145
Name: perc_change, dtype: float64

In [51]:
order_optimal_func(tmpdf)

1.553614231487775

In [47]:
order_optimal_func(tmpdf)

35764.632861090424